In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [ ]:
batch_size = 128
epochs = 10
cuda = False
seed = 1
log_interval = 1
lmbda = 10

In [ ]:
device = torch.device("cpu")
kwargs = {}

* Need to get the data
* Need to get the model to read in the data
* Need to get the model to have regression output
* Need the loss function to be L2 loss
* Need the loss function to be the rank function loss as well

# Below is a base case just to make sure things run

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

X = pd.read_csv("clusters2.csv")
X["label"] = X.index.values//50 + 1

In [ ]:
X = X[["Var1","Var2","Var3","Var4","label"]].as_matrix()
print(X[1:5,:5])

In [ ]:
#cov1 = np.diag([1 for i in range(4)])
#mu1 = np.array([10,10,0,0])
#mu2 = np.array([0,-5,0,-5])
#n=100
#X1 = np.random.multivariate_normal(mean=mu1,cov=cov1,size=n)
#X2 = np.random.multivariate_normal(mean=mu1[::-1],cov=cov1,size=n)
##X3 = np.random.multivariate_normal(mean=mu2,cov=cov1,size=n)
#X4 = np.random.multivariate_normal(mean=mu2[::-1],cov=cov1,size=n)
#X = np.vstack((X1,X2))
#X = np.vstack((X,X3))
#X = np.vstack((X,X4))
#labels = X[:,-1]
#indices = X[:,0]
#print(X[:,1:-1].shape)
#X = X[:,1:-1]
X = np.take(X,np.random.rand(X.shape[0]).argsort(),axis=0,out=X)
#X = X.astype(float)
labels = X[:,4]
X = X[:,:4]
my_x = [np.array(x1) for x1 in X]

train_loader = torch.utils.data.DataLoader(my_x)
test_loader = torch.utils.data.DataLoader(my_x)

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(4, 20).double()
        self.fc21 = nn.Linear(20, 2).double()
        self.fc22 = nn.Linear(20, 2).double()
        self.fc3 = nn.Linear(2, 20).double()
        self.fc4 = nn.Linear(20, 4).double()

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return self.fc4(h3)

    def forward(self, x):
        mu, logvar = self.encode(x.view(4))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar



In [ ]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
from scipy.stats import rankdata
d0 = np.array(euclidean_distances(X))
#d1 = np.array(euclidean_distances(X_hat2))

In [ ]:
for i in range(800):
    d0[:,i] = rankdata(d0[:,i])
#    d1[:,i] = rankdata(d1[:,i])

In [ ]:
a = 6
print(isinstance(a,int))

In [ ]:
n=800

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, ind, mu, logvar,epoch):
    l2 = torch.nn.MSELoss()
    loss2 = l2(recon_x.view(-1,4),x.view(-1,4))
    
    constraint = 0
    r = d0[:,ind]
    
    if epoch>1000000:
        for i in range(2,n):
            for j in range(i+1,n):
                try:
                    ind1 = np.where(r==i)[0][0]
                except:
                    ind1 = np.where(r==i)[0]
                try:
                    ind2 = np.where(r==j)[0][0]
                except:
                    ind2 = np.where(r==j)[0]
                try:
                    mu0,logvar0 = model.encode(datas[ind])
                    mu1,logvar1 = model.encode(datas[ind1])
                    mu2,logvar2 = model.encode(datas[ind2])
                    constraint += lmbda * (sum((mu0.detach().numpy()[0] - mu1.detach().numpy()[0])**2)<sum((mu0.detach().numpy()[0] - mu2.detach().numpy()[0])**2))
                except:
                    pass
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return loss2 + KLD + constraint


In [ ]:
datas = []
for batch_idx,data in enumerate(train_loader):
    datas.append(data)

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, batch_idx, mu, logvar,epoch)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))
    
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))
    return(train_loss)

In [ ]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
import time

In [ ]:
#train_loss = []
start = time.time()
for epoch in range(1, 4000 + 1):
    tl = train(epoch)
    train_loss.append(tl)
end = time.time()
print(end-start)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(train_loss)
plt.title(min(train_loss))

In [ ]:
mus =[]
logvars = []
X_hat = []
for i, data in enumerate(train_loader):
    data = data.to(device)
    mu, logvar = model.encode(data)
    mus.append(mu)
    logvars.append(logvar)
    recon_batch, mu, logvar = model(data)
    X_hat.append(recon_batch)

In [ ]:
points = np.array([0,0])
for point in mus:
    p = point.detach().numpy()[0]
    points = np.vstack((points,p))

In [ ]:
plt.rcParams["image.cmap"] = "tab20"
plt.scatter(points[1:,0],points[1:,1],c=labels)
plt.title("mu")

In [ ]:
points = np.array([0,0])
for point in logvars:
    p = point.detach().numpy()[0]
    points = np.vstack((points,p))
plt.rcParams["image.cmap"] = "tab20"
plt.scatter(points[1:,0],points[1:,1],c=labels)
plt.title("logvar")

In [ ]:
points = np.array([0,0,0,0])
for point in X_hat:
    p = point.detach().numpy()
    points = np.vstack((points,p))

In [ ]:
X_hat2 = points[1:,]
print(np.mean((X_hat2-X)**2))

In [ ]:
print(X_hat2[1:3,])
print(X[1:3,])
print(X_hat2[1:3,] - X[1:3,])

In [ ]:
np.max(X[:,1])